In [16]:
Pkg.Registry.add(Pkg.RegistrySpec(url = "$(Pkg.pkg_server())/registry/General", uuid = "23338594-aafe-5451-b93e-139f81909106"))

In [19]:
import Pkg;
Pkg.add("QuickHull")

LoadError: The following package names could not be resolved:
 * QuickHull (not found in project, manifest or registry)

## Problem1

In [22]:
struct SimplexState
    c::Vector
    A::Matrix
    b::Vector
    bset::Vector{Int} # columns of the BFP
    end

In [23]:
struct SimplexPoint
    x::Vector #
    binds::Vector{Int}
    ninds::Vector{Int}
    lam::Vector # equality Lagrange mults
    sn::Vector # non-basis Lagrange mults
    B::Matrix # the set of basic cols
    N::Matrix # the set of non-basic cols
    end
    # These are constructors for
    function SimplexPoint(T::Type)
    return SimplexPoint(zeros(T,0),zeros(Int,0),zeros(Int,0),
    zeros(T,0), zeros(T,0), zeros(T,0), zeros(T,0))
    end
    function SimplexPoint(T::Type, B::Matrix, N::Matrix)
    return SimplexPoint(zeros(T,0),zeros(Int,0),zeros(Int,0),
    zeros(T,0), zeros(T,0), B, N)
    end

    function simplex_point(state::SimplexState)
        m,n = size(state.A)
        @assert length(state.bset) == m "need more indices to define a BFP"
        binds = state.bset # basic variable indices
        ninds = setdiff(1:size(A,2),binds) # non-basic
        B = state.A[:,binds]
        N = state.A[:,ninds]
        cb = state.c[binds]
        cn = state.c[ninds]
        c = state.c
        @show cn
        if rank(B) != m
        return (:Infeasible, SimplexPoint(eltype(c), B, N))
        end
        xb = B\b
        x = zeros(eltype(xb),n)
        x[binds] = xb
        x[ninds] = zeros(eltype(xb),length(ninds))
        lam = B'\cb
        sn = cn - N'*lam
        @show sn
        if any(xb .< 0)
        return (:Infeasible, SimplexPoint(x, binds, ninds, lam, sn, B, N))
        else
        if all(sn .>= 0)
        return (:Solution, SimplexPoint(x, binds, ninds, lam, sn, B, N))
        else
        return (:Feasible, SimplexPoint(x, binds, ninds, lam, sn, B, N))
        end
        end
        end
    
        function simplex_step!(state::SimplexState)
            # get the current point from the new basis
            stat,p::SimplexPoint = simplex_point(state)
            if stat == :Solution
            return stat, p
            elseif state == :Infeasible
            return :Breakdown, p
            else # we have a BFP
            #= This is the Simplex Step! =#
            # take the Dantzig index to add to basic
            qn = findmin(p.sn)[2]
            q = p.ninds[qn] # translate index
            # check that nothing went wrong
            @assert all(state.A[:,q] == p.N[:,qn])
            d = p.B \ state.A[:,q]
            #@show d
            # TODO, implement an anti-cycling method /
            # check for stagnation and lack of progress
            # this checks for unbounded solutions
            if all(d .<= eps(eltype(d)))
            return :Degenerate, p
            end
            # determine the index to remove
            xq = p.x[p.binds]./d
            @show xq
            ninds = d .< eps(eltype(xq))
            xq[d .< eps(eltype(xq))] .= Inf
            pb = findmin(xq)[2]
            pind = p.binds[pb] # translate index
            @show p.binds, pb, pind, state.bset, q
            # remove p and add q
            @assert state.bset[pb] == pind
            state.bset[pb] = q
            return stat, p
            end
            end

simplex_step! (generic function with 1 method)

In [14]:
using LinearAlgebra
A1 = [1 1;
2 1 / 2;
]
b = [5; 8]
A = [A1 Matrix{Float64}(I,2,2)] # form the problem with slacks.
include("plotregion.jl")
PlotRegion.plotregion(A,b)

LoadError: LoadError: ArgumentError: Package QuickHull not found in current path.
- Run `import Pkg; Pkg.add("QuickHull")` to install the QuickHull package.
in expression starting at /Users/zhaorunchen/Public/Purdue/Spring_2023/CS 520/homework/hw6/plotregion.jl:1

In [24]:
teams = ["duke","miami","unc","uva","vt"]
data = [ # team 1 team 2, team 1 pts, team 2 pts
1 2 7 52 # duke played Miami and lost 7 to 52
1 3 21 24 # duke played unc and lost 21 to 24
1 4 7 38
1 5 0 45
2 3 34 16
2 4 25 17
2 5 27 7
3 4 7 5
3 5 3 30
4 5 14 52
]
ngames = size(data,1)
nteams = length(teams)
G = zeros(ngames, nteams)
p = zeros(ngames, 1)
for g=1:ngames
i = data[g,1]
j = data[g,2]
Pi = data[g,3]
Pj = data[g,4]
G[g,i] = 1
G[g,j] = -1
p[g] = Pi - Pj
end
A1 = [G -G Matrix{Float64}(-I,10,10);
-G G Matrix{Float64}(-I,10,10)]
A = [A1 Matrix{Float64}(I,20,20)]
b = [vec(p);
-vec(p)]

20-element Vector{Float64}:
 -45.0
  -3.0
 -31.0
 -45.0
  18.0
   8.0
  20.0
   2.0
 -27.0
 -38.0
  45.0
   3.0
  31.0
  45.0
 -18.0
  -8.0
 -20.0
  -2.0
  27.0
  38.0

In [25]:
function runsimplex(state::SimplexState)
    @show state
    status, p = simplex_step!(state)
    iter = 1
    while status != :Solution
    @show state.bset
    @show p.x
    status, p = simplex_step!(state)
    iter += 1
    end
    return p, state
    end
    function simplex_init(c,A,b)
    # need to get an initial BFP
    # setup the LP for phase 1
    m,n = size(A)
    e = sign.(b)
    cc = [zeros(n); ones(m)]
    CA = [A Diagonal(e)]
    cb = b
    bset = collect(n+1:n+m) # the last columns of cc
    phase1 = SimplexState(cc,CA,cb,bset)
    p, state = runsimplex(phase1)
    end

simplex_init (generic function with 1 method)

In [26]:
# Find startstate
ptstart, startstate = simplex_init([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],A,b)

state = SimplexState([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0 -1.0 0.0 0.0 0.0 -1.0 1.0 -0.0 -0.0 -0.0 -1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 -1.0 0.0 0.0 -1.0 -0.0 1.0 -0.0 -0.0 0.0 -1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 -1.0 0.0 -1.0 -0.0 -0.0 1.0 -0.0 0.0 0.0 -1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.0 0

(SimplexPoint([0.0, 20.0, 2.0, 0.0, 38.0, 31.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [36, 6, 12, 5, 11, 2, 27, 3, 14, 19, 21, 22, 23, 24, 15, 16, 17, 18, 29, 30], [1, 4, 7, 8, 9, 10, 13, 20, 25, 26, 28, 31, 32, 33, 34, 35, 37, 38, 39, 40], [-0.0, -0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0 -1.0 … 0.0 0.0; 0.0 -1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; -0.0 -0.0 … 1.0 0.0; -0.0 -1.0 … 0.0 1.0]), SimplexState([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0 -1.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; -0.0 -0.0 … 1.0 0.0; -0.0 -0.0 … 0.0 1.0], [-45.0, -3.0, -31.0, -45.0, 18.0, 8.0, 20.0, 2.0, -27.0, -38.0, 45.0, 3.0, 31.0, 45.0, -18.0, -8.0, -20.0, -2.0, 27.0, 38.0], [36, 6, 

In [27]:
state = startstate
@show state.bset
status, p = simplex_step!(state)
iter = 1
while status != :Solution
@show state.bset
status, p = simplex_step!(state)
iter += 1
end

state.bset = [36, 6, 12, 5, 11, 2, 27, 3, 14, 19, 21, 22, 23, 24, 15, 16, 17, 18, 29, 30]
cn = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
sn = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [28]:
p.x[1:10]

10-element Vector{Float64}:
  0.0
 20.0
  2.0
  0.0
 38.0
 31.0
  0.0
  0.0
  0.0
  0.0

In [ ]:
using LinearAlgebra
using Plots
plotly(size = (800, 600));
include("plotregion.jl");

struct SimplexState
c::Vector
A::Matrix
b::Vector
bset::Vector{Int} # columns of the BFP
end

struct SimplexPoint
x::Vector
binds::Vector{Int}
ninds::Vector{Int}
lam::Vector # equality Lagrange mults
sn::Vector # non-basis Lagrange mults
B::Matrix # the set of basic cols
N::Matrix # the set of non-basic cols
end

# These are constructors for SimplexPoint
function SimplexPoint(T::Type)
    return SimplexPoint(zeros(T,0),zeros(Int,0),zeros(Int,0),
    zeros(T,0), zeros(T,0), zeros(T,0), zeros(T,0))
end

function SimplexPoint(T::Type, B::Matrix, N::Matrix)
    return SimplexPoint(zeros(T,0),zeros(Int,0),zeros(Int,0),
    zeros(T,0), zeros(T,0), B, N)
end
    
function simplex_point(s::SimplexState)
    m,n = size(state.A)
    @assert length(state.bset) == m "need more indices to define a BFP"
    binds = state.bset # basic variable indices
    ninds = setdiff(1:size(state.A,2),binds) # non-basic
    B = state.A[:,binds]
    N = state.A[:,ninds]
    cb = state.c[binds]
    cn = state.c[ninds]
    c = state.c
    # @show cn
    if rank(B) != m
        return (:Infeasible, SimplexPoint(eltype(c), B, N))
    end
    xb = B\state.b
    x = zeros(eltype(xb),n)
    x[binds] = xb
    x[ninds] = zeros(eltype(xb),length(ninds))
    lam = B'\cb
    sn = cn - N'*lam
    # @show sn
    if any(xb .< 0)
        return (:Infeasible, SimplexPoint(x, binds, ninds, lam, sn, B, N))
    else
    if all(sn .>= 0)
        return (:Solution, SimplexPoint(x, binds, ninds, lam, sn, B, N))
    else
        return (:Feasible, SimplexPoint(x, binds, ninds, lam, sn, B, N))
    end
end
end
    
function simplex_step!(state::SimplexState)
    # get the current point from the new basis
    stat,p::SimplexPoint = simplex_point(state)

    if stat == :Solution
        return (stat, p)
    elseif state == :Infeasible
        return (:Breakdown, p)
    else # we have a BFP
        #= This is the Simplex Step! =#
        # take the Dantzig index to add to basic
        qn = findmin(p.sn)[2]
        q = p.ninds[qn] # translate index
        # check that nothing went wrong
        @assert all(state.A[:,q] == p.N[:,qn])
        d = p.B \ state.A[:,q]
        #@show d
        # TODO, implement an anti-cycling method /
        # check for stagnation and lack of progress
        # this checks for unbounded solutions

        if all(d .<= eps(eltype(d)))
            return (:Degenerate, p)
        end

    # determine the index to remove
    xq = p.x[p.binds]./d
    ninds = d .< eps(eltype(xq))
    xq[d .< eps(eltype(xq))] .= Inf
    pb = findmin(xq)[2]
    pind = p.binds[pb] # translate index
    #@show p.binds, pb, pind, state.bset, q
    # remove p and add q
    @assert state.bset[pb] == pind
    state.bset[pb] = q
    return (stat, p)
end
end

function solve!(state::SimplexState)
    PlotRegion.plotregion(state.A, state.b);
    @show state.bset;
    status, p = simplex_step!(state);
    iter = 1;
    while status != :Solution
        @show state.bset;
        @show p.x;
        scatter!([p.x[1]],[p.x[2]],
        series_annotations=["$(iter)"],marker=(15,0.2,:orange),label="");
        status, p = simplex_step!(state);
        iter += 1;
    end
    @show state.bset;
    @show p.x;
    scatter!([p.x[1]],[p.x[2]],
        series_annotations=["$(iter)"],marker=(15,0.2,:red),label="")
end

# Problem 1: everything goes well
c1 = [-5.0; -1.0];
A1 = [1.0 1.0;
    2.0 0.5];
b1 = [5.0; 8.0];
A1_slacks = [A1 Matrix{Float64}(I,2,2)]
c1_slacks = [c1; 0.0; 0.0];

# Problem 2: unbounded, the solution is minus infinite
c2 = [-1.0; -3.0];
A2 = [-2.0 1.0;
    -1.0 2.0];
b2 = [2.0; 7.0];
A2_slacks = [A2 Matrix{Float64}(I,2,2)]
c2_slacks = [c2; 0.0; 0.0];

# Problem 3: degenerate, it is slower (we are already at the optimal point)
# The first BFP is optimal but the algorithm is not aware of that.
c3 =[-(3.0/4.0); 150.0; -(1.0/50.0); 6.0];
A3 = [0.25 -60.0 -(1.0/25.0) 9.0;
    0.50 -90.0 (1.0/50.0) 3.0;
    0.0 0.0 1.0 0.0];
b3 = [0.0; 0.0; 1.0];
A3_slacks = [A3 Matrix{Float64}(I,3,3)]
c3_slacks = [c3; 0.0; 0.0; 0.0];

# Start off with the point (0,0)
state = SimplexState(c1_slacks, A1_slacks, b1, [3; 4]);
solve!(state);